In [104]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [105]:
from pathlib import Path
ROOT = Path('/content/drive/MyDrive/universite-paris-saclay/nlp/rottentomatoes')

In [ ]:
!pip install transformers

In [107]:
from pathlib import Path
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.decomposition import PCA
from wordcloud import WordCloud
from transformers import BertModel, BertTokenizer
from sklearn.metrics import accuracy_score
from torch.utils.data import TensorDataset, Dataset, DataLoader, random_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler


import matplotlib.pyplot as plt
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import nltk
import gensim
import torch
import torch.nn as nn

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [ ]:
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")
nltk.download("omw-1.4")

In [109]:
reviews_df = pd.read_csv(ROOT / "regression_reviews.csv")
reviews_df.sample(10, random_state=2023)

,review_score,review_content
7958,4.0,robbie janney flawless compelling corrective a...
7513,3.5,told solid timeless craftsmanship brickie prot...
3105,1.5,get drunk solemn resonance turn intriguing ele...
1924,0.5,tacky cgidriven actionhorror flick vacuous wri...
4589,2.0,homegrown appeal streetdance get lost sequel o...
9689,5.0,sorrentino may well made masterpiece great bea...
9806,5.0,one delightful surprise lost none charm surpri...
2227,1.0,like marshall last ensemble piece valentine da...
3679,1.5,nearbrilliance robert rodriguez achieved sin c...
6119,3.0,call kind vaguely smart extremely gory gloop w...


In [110]:
# First we need to rescale the values from [0,5] to [0,1]
reviews_df['review_score'] = reviews_df['review_score'] / 5.0
reviews_df.sample(10, random_state=2023)

,review_score,review_content
7958,0.8,robbie janney flawless compelling corrective a...
7513,0.7,told solid timeless craftsmanship brickie prot...
3105,0.3,get drunk solemn resonance turn intriguing ele...
1924,0.1,tacky cgidriven actionhorror flick vacuous wri...
4589,0.4,homegrown appeal streetdance get lost sequel o...
9689,1.0,sorrentino may well made masterpiece great bea...
9806,1.0,one delightful surprise lost none charm surpri...
2227,0.2,like marshall last ensemble piece valentine da...
3679,0.3,nearbrilliance robert rodriguez achieved sin c...
6119,0.6,call kind vaguely smart extremely gory gloop w...


In [111]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [112]:
# Create a function to tokenize a set of texts
def preprocessing_for_bert(data, max_len):
    """Perform required preprocessing steps for pretrained BERT.
    @param    data (np.array): Array of texts to be processed.
    @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
    @return   attention_masks (torch.Tensor): Tensor of indices specifying which
                  tokens should be attended to by the model.
    """
    # Create empty lists to store outputs
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in data:
        # `encode_plus` will:
        #    (1) Tokenize the sentence
        #    (2) Add the `[CLS]` and `[SEP]` token to the start and end
        #    (3) Truncate/Pad sentence to max length
        #    (4) Map tokens to their IDs
        #    (5) Create attention mask
        #    (6) Return a dictionary of outputs
        encoded_sent = tokenizer.encode_plus(
            text = sent,  # Preprocess sentence
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=max_len,                  # Max length to truncate/pad
            pad_to_max_length=True,         # Pad sentence to max length
            #return_tensors='pt',           # Return PyTorch tensor
            return_attention_mask=True      # Return attention mask
            )
        
        # Add the outputs to the lists
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

In [119]:
# Sample just the 10% of the data
perc = 0.3
reviews_df = reviews_df.sample(frac=perc, random_state=2023)

X = reviews_df['review_content'].values
y = reviews_df['review_score'].values

X_train, X_test, y_train, y_test =\
    train_test_split(X, y, test_size=0.1, random_state=2023)

X_train, X_val, y_train, y_val =\
    train_test_split(X_train, y_train, test_size=0.1, random_state=2024)

# Tokenize the input text

max_len = 30

# Run function `preprocessing_for_bert` on the train set, the validation set and the test set
train_inputs, train_masks = preprocessing_for_bert(X_train, max_len)
val_inputs, val_masks = preprocessing_for_bert(X_val, max_len)
test_inputs, test_masks = preprocessing_for_bert(X_test, max_len)

# Convert other data types to torch.Tensor
train_labels = torch.tensor(y_train)
val_labels = torch.tensor(y_val)
test_labels = torch.tensor(y_test)

# For fine-tuning BERT, it's recommended a batch size of 16 or 32.
batch_size = 16

# Create the DataLoader for our training set
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set
val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

# Create the DataLoader for our test set
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=1)

In [114]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla T4


In [115]:
class BertForRegression(nn.Module):
    def __init__(self):
        super(BertForRegression, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.1)
        self.linear = nn.Linear(self.bert.config.hidden_size, 1)
        self.double()

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]  # Use the second output (pooled output)
        pooled_output = self.dropout(pooled_output)
        regression_output = self.linear(pooled_output)
        return regression_output.squeeze(-1)

In [116]:
def train(model, train_dataloader, optimizer, criterion, device):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(train_dataloader):
        input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)
        # input_ids = batch['input_ids'].to(device)
        # attention_mask = batch['attention_mask'].to(device)
        # labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(train_dataloader)


def evaluate(model, val_dataloader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(val_dataloader):
            input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)

            outputs = model(input_ids, attention_mask=attention_mask)
            loss = criterion(outputs, labels)

            epoch_loss += loss.item()

    return epoch_loss / len(val_dataloader)


In [120]:
model = BertForRegression()
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
criterion = nn.MSELoss()

train_losses = []
# train_accs = []
val_losses = []
# val_accs = []

num_epochs = 5

for epoch in range(num_epochs):
    train_loss = train(model, train_dataloader, optimizer, criterion, device)
    train_losses.append(train_loss)
    val_losses.append(train_loss)
    val_loss = evaluate(model, val_dataloader, criterion, device)
    print(f"Epoch {epoch+1}. Train loss: {train_loss:.5f}. Validation loss: {val_loss:.5f}")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch 1. Train loss: 0.13284. Validation loss: 0.10035
Epoch 2. Train loss: 0.09490. Validation loss: 0.07012
Epoch 3. Train loss: 0.05790. Validation loss: 0.07721
Epoch 4. Train loss: 0.04728. Validation loss: 0.06238
Epoch 5. Train loss: 0.02902. Validation loss: 0.06602


In [121]:
torch.save(model.state_dict(), ROOT / "bert-rottentomatoes-regression-03-v2.pth")

In [42]:
class BertForRegression(nn.Module):
    def __init__(self):
        super(BertForRegression, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.1)
        self.linear = nn.Linear(self.bert.config.hidden_size, 1)
        self.double()

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]  # Use the second output (pooled output)
        pooled_output = self.dropout(pooled_output)
        regression_output = self.linear(pooled_output)
        return regression_output.squeeze(-1)

In [ ]:
model = BertForRegression()
model.load_state_dict(torch.load(ROOT / "bert-rottentomatoes-regression-01-v1.pth"))
model.to(device)

In [53]:
def predict(model, test_dataloader):
    model.eval()
    predictions = []
    with torch.no_grad():
        for batch in test_dataloader:
            input_ids, attention_mask, _ = tuple(t.to(device) for t in batch)
            # input_ids = batch['input_ids'].to(device)
            # attention_mask = batch['attention_mask'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            predictions.extend(outputs.tolist())

    return predictions

In [58]:
predictions = predict(model, test_dataloader)
targets = y_test

mse = mean_squared_error(targets, predictions)
rmse = mean_squared_error(targets, predictions, squared=False)
mae = mean_absolute_error(targets, predictions)
r2 = r2_score(targets, predictions)

print(f"MSE: {mse:.5f}")
print(f"RMSE: {rmse:.5f}")
print(f"MAE: {mae:.5f}")
print(f"R2: {r2:.5f}")

MSE: 0.11688
RMSE: 0.34187
MAE: 0.29769
R2: -0.04755


In [86]:
def custom_test(model, text):
    input = tokenizer(text, return_tensors='pt')

    # Make a prediction with the model
    output = model(input['input_ids'], input['attention_mask'])
    return output.item() * 5.0

In [61]:
def preprocess_text(text):
    # Removing numerical data or irrelevant characters
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    # Tokenization
    tokens = nltk.word_tokenize(text)
    # Lowercasing
    tokens = [token.lower() for token in tokens]
    # Removing stopwords
    tokens = [token for token in tokens if token not in stopwords.words("english")]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    # Joining the tokens back to a string
    text = " ".join(tokens)
    return text

In [60]:
model = BertForRegression()
model.load_state_dict(torch.load(ROOT / "bert-rottentomatoes-regression-01-v1.pth"))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [87]:
reviews = pd.read_csv(ROOT / "clean_reviews.csv")
reviews = reviews.sample(20, random_state=2023)

for r in reviews.iterrows():
    _, r = r
    review_type, review_score, review_content = r
    # print(r)eview_type, review_score, review_content = r
    print('REVIEW:')
    print(review_content)
    print('True score:', review_score)
    print('Prediction:', custom_test(model, preprocess_text(review_content)))

REVIEW:
THE WORST FILM OF THE YEAR.
True score: 0.0
Prediction: 3.260732027381536
REVIEW:
Offers creative action, a deep story, gorgeously colorful visuals, and enough characters to grow and love.
True score: 3.5
Prediction: 1.8287923215176005
REVIEW:
The film takes some deciphering, but once a viewer cracks its code "Alps" opens up into something expansive and rich.
True score: 4.0
Prediction: 1.9253423299907708
REVIEW:
While Lola is adorable throughout, neither she nor the film will be remembered the morning after.
True score: 3.0
Prediction: 2.475978165896014
REVIEW:
What's left is a sequence of downbeat sex skits, slotted together with all the finesse of an Ikea cabinet.
True score: 2.0
Prediction: 2.948368115636786
REVIEW:
Aamir Khan is stupendous as the rule-breaker Rancho. But the rest of the cast doesn't remain in the shadows.
True score: 5.0
Prediction: 2.165789137607041
REVIEW:
Paul Walter Hauser is the only reason to see this film. What we are witnessing with Hauser is the e

In [138]:
# ============================================== #
# =============== Manual Testing =============== #
# ============================================== #


In [139]:
# CLASSIFICATION

class BertForSentimentAnalysis(nn.Module):
    def __init__(self):
        super(BertForSentimentAnalysis, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(768, 2) # 768 is the output dimension of BERT model
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

In [140]:
# REGRESSION

class BertForRegression(nn.Module):
    def __init__(self):
        super(BertForRegression, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.1)
        self.linear = nn.Linear(self.bert.config.hidden_size, 1)
        self.double()

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]  # Use the second output (pooled output)
        pooled_output = self.dropout(pooled_output)
        regression_output = self.linear(pooled_output)
        return regression_output.squeeze(-1)

In [141]:
model_classification = BertForSentimentAnalysis()
model_classification.load_state_dict(torch.load(ROOT / "bert-rottentomatoes-binary-100-v4.pth"))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [142]:
model_regression = BertForRegression()
model_regression.load_state_dict(torch.load(ROOT / "bert-rottentomatoes-regression-03-v2.pth"))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [143]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [144]:
def preprocess_text(text):
    # Removing numerical data or irrelevant characters
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    # Tokenization
    tokens = nltk.word_tokenize(text)
    # Lowercasing
    tokens = [token.lower() for token in tokens]
    # Removing stopwords
    tokens = [token for token in tokens if token not in stopwords.words("english")]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    # Joining the tokens back to a string
    text = " ".join(tokens)
    return text

In [145]:
def custom_classification_test(model, text):
    inputs = tokenizer(text, return_tensors='pt')

    # Make a prediction with the model
    outputs = model(inputs['input_ids'], inputs['attention_mask'])
    predicted_class = torch.argmax(outputs[0])

    # Print the predicted sentiment
    return 'Fresh' if predicted_class == 1 else 'Rotten'

In [146]:
def custom_regression_test(model, text):
    input = tokenizer(text, return_tensors='pt')
    output = model(input['input_ids'], input['attention_mask'])
    return output.item() * 5.0

In [148]:
reviews = pd.read_csv(ROOT / "clean_reviews.csv")
reviews = reviews.sample(50, random_state=2025)

for r in reviews.iterrows():
    _, r = r
    review_type, review_score, review_content = r
    # print(r)eview_type, review_score, review_content = r
    print('REVIEW:')
    print(review_content)

    text = preprocess_text(review_content)

    print('True type:', 'Rotten' if review_type == 0 else 'Fresh')
    print('Predicted type:', custom_classification_test(model_classification, text))

    print('True score:', review_score)
    print('Predicted score:', custom_regression_test(model_regression, text))


REVIEW:
A pedal-to-the-metal action adventure - with very little under the hood.
True type: Rotten
Predicted type: Rotten
True score: 2.5
Predicted score: 3.579523364042622
REVIEW:
Any resemblence to Clive Barker's real work is purely coincidental.
True type: Rotten
Predicted type: Rotten
True score: 0.0
Predicted score: 2.742251743557361
REVIEW:
Two reasons not to go see Abduction: first, every screening is guaranteed to be jam-packed with rabid girls squealing every time its star, Twilight's Taylor Lautner is on screen. Second: it's just not a good movie.
True type: Rotten
Predicted type: Rotten
True score: 1.0
Predicted score: 1.1469731499778335
REVIEW:
It underlines the grand feats the human body is capable of and how far it can be stretched through a crazy, nerve-shredding expedition.
True type: Fresh
Predicted type: Fresh
True score: 3.5
Predicted score: 4.110241019940163
REVIEW:
As misguided a film as you're likely to see from a genuinely great filmmaker.
True type: Rotten
Predi